In [ ]:
!pip install --upgrade --quiet openai

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
#BERT:
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
#Random forest:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
#FFN
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
#Multimodal ML
import re
from sklearn.feature_selection import SelectKBest, f_regression
#Explainable ML
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
trancripts_csv = pd.read_csv("transcripts.csv", header=None, names=["Participant", "Transcripts"])
scores_csv = pd.read_csv("scores.csv")

In [ ]:
import pandas as pd
import re, string, nltk
from nltk import word_tokenize
nltk.download('punkt', quiet=True)

df_merge = pd.merge(trancripts_csv, scores_csv, on="Participant", how='inner')

# Minimal text‑cleaning function --------------------------------------------
def clean(text: str) -> str:
    text = text.lower()
    text = re.sub(r"\d+", " ", text)           # remove digits
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"\s{2,}", " ", text)        # squeeze spaces
    return text.strip()

df_merge["Transcripts"] = df_merge["Transcripts"].apply(clean)

In [ ]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Paste your OpenAI key: ")

from openai import OpenAI
client = OpenAI()

Paste your OpenAI key:  ········


In [ ]:
import os, re, json, random, time
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import pearsonr
from sklearn.model_selection import GroupKFold
from openai import OpenAI

# ------------------------------------------------------------
# 2.1  Load your dataframe (⇐ already in memory for you)
# ------------------------------------------------------------
df = df_merge.copy()            # columns: Participant, Transcripts, Overall, Excited
groups = df["Participant"]

# ------------------------------------------------------------
# 2.2  OpenAI client
# ------------------------------------------------------------
client = OpenAI()               # reads OPENAI_API_KEY from env
MODEL = "gpt-4.1-mini"           # or "gpt-4.1-mini" if enabled for your org

# ------------------------------------------------------------
# 2.3  Prompt template
# ------------------------------------------------------------
TEMPLATE = """You are an HR expert.\nGiven an interview transcript, rate:\n• overall performance (1–7)\n• excitement/enthusiasm (1–7)\nReturn strict JSON: {{\"overall\": <float>, \"excitement\": <float>, \"explanation\": \"<why>\"}}\n{fewshot}\n<TRANSCRIPT>\n{transcript}\n</TRANSCRIPT>\nJSON:
"""

def make_fewshot(df_sub, n=2):
    """Pick n random labelled examples to include in the prompt."""
    rows = random.sample(list(df_sub.index), n)
    shots = []
    for idx in rows:
        row = df_sub.loc[idx]
        shots.append(
f"""Example:
<TRANSCRIPT>
{row['Transcripts'][:600]}
</TRANSCRIPT>
{{"overall": {row['Overall']:.1f}, "excitement": {row['Excited']:.1f}, "explanation": "sample"}}"""
        )
    return "\n".join(shots)

# ------------------------------------------------------------
# 2.4  Single‑call helper
# ------------------------------------------------------------
def gpt_score(prompt):
    """Returns (overall, excitement, explanation) or None."""
    try:
        rsp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=120,
            response_format={"type": "json_object"}   # forces JSON output
        )
        js = json.loads(rsp.choices[0].message.content)
        return float(js["overall"]), float(js["excitement"]), js["explanation"]
    except Exception as e:
        print("⚠️  GPT call failed:", e)
        return None

# ------------------------------------------------------------
# 2.5  5‑fold participant‑level CV
# ------------------------------------------------------------
fold = GroupKFold(n_splits=5)
records = []

for f, (tr, te) in enumerate(fold.split(df, groups=groups)):
    fewshot = make_fewshot(df.iloc[tr], n=2)

    for idx in tqdm(te, desc=f"Fold {f+1}/5"):
        txt = df.at[idx, "Transcripts"][:1800]      # keep cost low
        prompt = TEMPLATE.format(fewshot=fewshot, transcript=txt)
        res = gpt_score(prompt)
        if res is None:
            continue
        o_hat, e_hat, expl = res
        records.append({
            "idx": idx, "fold": f,
            "o_hat": o_hat, "e_hat": e_hat,
            "o_true": df.at[idx, "Overall"],
            "e_true": df.at[idx, "Excited"],
            "explanation": expl
        })
        time.sleep(0.5)   # polite rate‑limit; tweak as needed

out = pd.DataFrame(records)
print(f"\nCollected {len(out)} / {len(df)} predictions")

# ------------------------------------------------------------
# 2.6  Metrics
# ------------------------------------------------------------
def rel_err(y, yhat, max_score=7):
    return np.mean(np.abs(yhat - y)) / max_score

r_overall = pearsonr(out["o_true"], out["o_hat"])[0]
re_overall = rel_err(out["o_true"], out["o_hat"])
r_excited = pearsonr(out["e_true"], out["e_hat"])[0]
re_excited = rel_err(out["e_true"], out["e_hat"])

print(f"GPT‑4.1‑mini results (5‑fold):")
print(f"  Overall   r = {r_overall:.3f}   RE = {re_overall:.3f}")
print(f"  Excitement r = {r_excited:.3f}   RE = {re_excited:.3f}")


Fold 5/5: 100%|█████████████████████████████████| 27/27 [01:00<00:00,  2.25s/it]


Collected 138 / 138 predictions
GPT‑4o‑mini results (5‑fold):
  Overall   r = 0.484   RE = 0.070
  Excitement r = 0.353   RE = 0.104


In [ ]:
records

[]